# Gramatica este totul. Corectarea erorilor gramaticle


## Introducere

Corectarea erorilor gramaticale (GEC) se ocupă de a corecta diferite tipuri de erori din text, cum ar fi erorile de ortografie, de punctuație sau gramaticale.
Un sistem GEC primește la intrare o propoziție potențial eronată și este de așteptat să o transforme în versiunea sa corectată.

Voi trebuie sa contruiți un sistem GEC adaptat pentru limba română. Printre posibilele greșeli în limba română se regăsesc: lipsa diacriticelor, greșeli ortografice de una sau două litere per cuvânt, prepoziții incorecte, dezacorduri de gen/număr/caz/persoană, cacofonii sau greșeli de punctuație.

Căteva exemple de texte cu greșeli și variantele corectate:

* Exemplu 1:

eu am duar 10 ani si stiu melodia asta de la 7 anieste preferata mea e frumoasa si eu stiu ca te chinui mult **->** Eu am doar 10 ani și știu melodia asta de la 7 ani, este preferata mea, e frumoasă și eu știu că te chinui mult.


* Exemplu 2:

Biatul acesta este special,vocea lui,cred ca toi am avut acest gand **->** Băiatul acesta este special, vocea lui, cred că toți am avut acest gând.

* Exemplu 3:

Am mai auzit melodi dar asta este cea mai bun **->** Am mai auzit melodii dar asta este cea mai bună.


## Obiectiv

Scopul este să construiți cel mai performant model de tip GEC pentru limba română, operând sub următoarele restricții:

*   Modelul trebuie sa fie de tip encoder-decoder (de exemplu, bazat pe mBART sau mT5).
*   Folosiți doar variante "base" ale modelelor (unde acestea există).
*   Nu aveți voie să folosiți date deja generate de alte entități pentru GEC sau modele deja finetunate pentru asta.
*   Pentru antrenare aveti acces **doar** la datele pe care vi le punem la dispoziție. Acestea sunt texte care provin din articole de pe Wikipedia în română și le vom considera corecte. Aveți voie să le folosiți în orice fel doriți, să le alterați în orice mod considerați benefic. Nu aveți voie să folosiți alte texte!

## Sfaturi

* Pornind de la date curate, încercați să vă generați automat date de antrenare.
* Evaluarea se va face pe un set divers de propoziții, dar la nivel de propoziție. Astfel, la evaluare va trebui ca modelul vostru să corecteze câte o propoziție pe rând.



## Livrabile

Trebuie să submiteți următoarele:

*   Un model încărcat pe Huggingface Hub (vezi parametrul push_to_hub; alternativ puteți încărca modelul direct de pe Huggingface, din browser).
*   Un raport tehnic de maxim două pagini în care să explicați cum ați rezolvat problema. Raportul poate fi scris în engleza sau în română.


## Cerințe preliminare



### Configurație HuggingFace

Înainte de a incepe propriu-zis rezolvarea problemei trebuie să:

1. Intrați pe pagina de [HuggingFace](https://huggingface.co/Olimpiada-AI) și cereți acces la date.

2. În setări, creați Access Tokens, unul pentru "read" și unul pentru "write" și salvați-le în [Colab Secrets](https://www.youtube.com/watch?v=q87i2LZbbPc) ca `hf_read` și `hf_write`.

In [ ]:
from google.colab import userdata

read_access_token = userdata.get('hf_read')
write_access_token = userdata.get('hf_write')

### Module necesare

In [ ]:
import importlib
import torch, transformers


if '2.3.0' not in torch.__version__:
  !pip install torch==2.3.0
if transformers.__version__!='4.41.2':
  !pip install transformers==4.41.2

if importlib.util.find_spec('datasets') is None:
  !pip install datasets==2.18.0
  !pip install evaluate==0.4.2
  !pip install accelerate -U

!pip install rouge_score

In [ ]:
!pip install python-Levenshtein

import importlib
import torch
import transformers
import random
import Levenshtein
import textwrap
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

Dacă tocmai ați instalat `accelerate`, executați `Runtime > Restart session and run all` din meniul Colab.

# Date

In [ ]:
import textwrap

def split_paragraph(example):
    return {"part": [part for foo in example["page"] for part in textwrap.wrap(foo, 100)]}

# Abordare generala a problemei

Problema respectiva ne cere sa creem un corector de typos care sa performeze pentru limba romana. Pornind de la datele curate de antrenare, pentru a simula cat mai bine erorile comune ale limbii romane, am introdus erori comune ale limbii romane: spelling, punctuatie, diacritice, conjugari gresite si propozitii care nu sunt la locul lor. Scopul principal ar fi sa generam din date curate cat mai multe exemple pe care sa se antreneze modelul, exemple cu erori clasice ale limbii romane. La greseli comune ale limbii romane, ne putem concentra pe mai multe:

## Generare de erori comune ale limbii romane

Pentru spelling, facem o schimbare random a unui caracter dintr-un cuvant, simuland un keystroke prost de la tastatura.

Pentru punctuatie, introducem sau scoatem punctuatia la finalul unei propozitii din setul nostru de date.

Diacriticile specifice limbii romane sunt mapate cu echivalentul lor in limba engleza.

Pentru articol, schimbam articolul barbat cu articolul femeie. "barbatul -> barbata"

Prepozitiile le mai schimbam random dintr-o lista de prepozitii comune ale limbii romane.

Analog pentru conjugarile clasice ale verbelor in limba romana.

Nota: la final mi-am dat seama pe baza datelor de intrare ca poate era bine sa adaug si un caz in care sa pun o litera random la inceputul/finalul propozitiei.

In [ ]:
def introduce_advanced_errors(sentence):
    words = sentence.split()
    altered_words = []

    for word in words:
        altered_word = word

        if random.random() < 0.2:  # 20% sansa de a altera cuvantul
            error_type = random.choice(['spelling', 'punctuation', 'diacritics', 'agreement', 'preposition', 'conjugation'])

            if error_type == 'spelling':
                if len(word) > 3:  # doar daca lungimea cuvantului este mai mare decat 3, altfel s-ar pierde cuvantul ex: mar -> mir, ceea ce nu ne convine
                    altered_word = list(word)
                    idx = random.randint(0, len(altered_word) - 1)
                    altered_word[idx] = random.choice('abcdefghijklmnopqrstuvwxyz')  # la intamplare se da replace la un caracter din alfabet
                    altered_word = ''.join(altered_word)

            elif error_type == 'punctuation':
                if random.random() < 0.5:
                    altered_word = word + random.choice(['.', ',', '!', '?'])  # se adauga extra punctuatia
                else:
                    altered_word = word.strip('.,!?')  # se da remove la punctuatie

            elif error_type == 'diacritics':
                diacritics_map = {'ă': 'a', 'â': 'a', 'î': 'i', 'ș': 's', 'ț': 't', 'a': 'ă', 'a': 'â', 'i': 'î', 's': 'ș', 't': 'ț'}
                altered_word = ''.join([diacritics_map.get(char, char) for char in word])  # mapare diacritice cu echivalentul in engleza

            elif error_type == 'agreement':
                if word.endswith('ul'):
                    altered_word = word[:-2] + 'a'  # femeie, barbat
                elif word.endswith('a'):
                    altered_word = word[:-1] + 'ul'

            elif error_type == 'preposition':
                prepositions = ['la', 'de', 'pe', 'cu', 'din', 'pentru']
                if word in prepositions:
                    altered_word = random.choice(prepositions)  # replace cu diferite prepozitii din lista.

            elif error_type == 'conjugation':
                if word.endswith('e'):
                    altered_word = word[:-1] + 'i'  # exemple clasice de conjugari proaste
                elif word.endswith('i'):
                    altered_word = word[:-1] + 'e'

        altered_words.append(altered_word)

    return " ".join(altered_words)

In [ ]:
def augment_sentence(example):
    original_sentence = example["part"]
    altered_sentence = introduce_advanced_errors(original_sentence)
    return {"original_sentence": original_sentence, "altered_sentence": altered_sentence}

In [ ]:
# load the data
# boilerplate

import random

from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

wiki_dataset = load_dataset('Olimpiada-AI/ro_wiki', token=read_access_token)

# merge splits into a single dataset
wiki_dataset = concatenate_datasets([wiki_dataset["validation"], wiki_dataset["test"]])

# split into smaller chunks
wiki_dataset = wiki_dataset.map(split_paragraph, batched=True, remove_columns="page")

# alter sentences
wiki_dataset = wiki_dataset.map(augment_sentence, batched=False, remove_columns="part")

# split into train and validation
wiki_dataset= wiki_dataset.train_test_split(test_size=0.05)

# Tokenizare si impartirea datelor in date de antrenare si date de testare.

# Baseline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base") # se tokenizeaza si se impart datele in date de antrenare si de validare, doresc sa compar si mBart si mt5 de la google.
tokenizer_mbart = AutoTokenizer.from_pretrained("facebook/mbart-large-50")
max_length = 256

train_data = wiki_dataset['train'].map(lambda x: {'input_ids': tokenizer(x['altered_sentence'], truncation=True, max_length=max_length)["input_ids"], 'label': tokenizer(x['original_sentence'], truncation=True, max_length=max_length)["input_ids"]}, remove_columns=["original_sentence", "altered_sentence"])
val_data = wiki_dataset['test'].map(lambda x: {'input_ids': tokenizer(x['altered_sentence'], truncation=True, max_length=max_length)["input_ids"], 'label': tokenizer(x['original_sentence'], truncation=True, max_length=max_length)["input_ids"]}, remove_columns=["original_sentence", "altered_sentence"])

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/2033 [00:00<?, ? examples/s]

In [ ]:
# from transformers import AutoModelForSequenceClassification, TrainingArguments, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

# Metrice de evaluare

O idee foarte frumoasa pe care am incercat-o sa o implementez este utilizarea distantei Levenshtein. Distanta Levenshtein este utilizata si la autocorector la telefon pentru a vedea care cuvinte sunt cele mai apropiate de ce a dat type utilizatorul, ceea ce este foarte convenabil pentru situatia noastra actuala. Algoritmul se bazeaza pe tehnica programarii dinamice si este descris amanuntit intr-o lucrare publicata impreuna cu profesorul meu despre "String Manipulation" : https://www.amazon.com/Algorithmic-Efficiency-Handbook-High-Performance-Computing-ebook/dp/B0C9XK6ZCB/ref=sr_1_1?crid=3BC3Q6POXZOYE&dib=eyJ2IjoiMSJ9.6r3_uu1Eqm_YVZyvUmiFqQ.9uWDbsjepqS38NJ1z1alqqP5bi0K_T8nR_2fhPBa-Sc&dib_tag=se&keywords=miron+alexandru+bogdan&qid=1720778253&sprefix=miron+alexandru+bogdan%2Caps%2C297&sr=8-1 , motiv pentru care am ales proba de NLP in locul probei de Computer Vision, deoarece sunt mai familiarizat cu manipularea de caractere.

Dupa aceea , facem load la diferite metrice comune de Natural Language Processing, bleu, meteor si rouge. Am fost reticent sa utilizez si alte metrice, precum F1 score pentru acest task deoarece nu sunt familiar cu comportamentul acestora pe NLP, asa ca m-am rezumat la metodele clasice.

In [ ]:
# se defineste evaluation metrics, bleu , meteor si rouge.
bleu = evaluate.load("bleu")
meteor = evaluate.load('meteor')
rouge = evaluate.load('rouge')

def compute_edit_distance(predictions, references):
    total_distance = 0
    for pred, ref in zip(predictions, references):
        total_distance += Levenshtein.distance(pred, ref[0]) # se adauga la distanta totala distanta levenshtein. si se imparte la lungimea predictilor
    return total_distance / len(predictions)

def compute_metrics(eval_pred):
    raw_predictions, raw_labels = eval_pred
    predictions = []
    labels = []

    for pred in raw_predictions:
        pred = list(filter(lambda x: x != -100, pred))
        text_predictions = tokenizer.decode(pred, skip_special_tokens=True)
        predictions.append(text_predictions)

    for label in raw_labels:
        label = list(filter(lambda x: x != -100, label))
        text_labels = tokenizer.decode(label, skip_special_tokens=True)
        labels.append([text_labels])

    res_bleu = bleu.compute(predictions=predictions, references=labels)["bleu"]
    res_meteor = meteor.compute(predictions=predictions, references=labels)["meteor"]
    res_rouge = rouge.compute(predictions=predictions, references=labels)["rougeL"]
    edit_distance = compute_edit_distance(predictions, labels)
    return {"bleu": res_bleu, "meteor": res_meteor, "rouge-L": res_rouge, "edit_distance": edit_distance}


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Cross validation pentru mBart si pentru mt5

Ca si la taskurile clasice de machine learning, facem crossvalidation in 5 "folduri" pentru a ne asigura ca modelul nostru generalizeaza bine pe seturi noi de date pe care acesta nu le-a mai vazut. Asa putem vedea si first hand cam care model, dupa preprocesarea datelor, mBart sau mt5, performeaza mai bine si generalizeaza mai bine pentru taskul nostru de typo correction in limba romana.
Pana la urma am observat ca mt5 performeaza mai bine decat mBart, asa ca pe acesta l-am folosit pana la urma. Mai mult, cautand pe google am vazut cateva articole care scrie ca mt5 este mai bun pentru text2text task, ceea ce imi confirma judecata si experimentarea.Urmatorul pas este gasirea parametrilor optimi utilizand GridSearch CV.

In [ ]:
# cross validation pe un split de 5 folduri
kf = KFold(n_splits=5)

# convertire la o lista pentru indexare.
dataset_list = wiki_dataset['train'].map(lambda x: {'altered_sentence': x['altered_sentence'], 'original_sentence': x['original_sentence']})
dataset_list = [dataset_list[i] for i in range(len(dataset_list))]

model_scores = {"mT5": [], "mBART": []}

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset_list)):
    print(f"Fold {fold + 1}")
    train_split = [dataset_list[i] for i in train_idx]
    val_split = [dataset_list[i] for i in val_idx]

    for model_name, model_tokenizer, model_id in [("mT5", tokenizer_mt5, "google/mt5-base"), ("mBART", tokenizer_mbart, "facebook/mbart-large-50")]:
        print(f"Training {model_name} for fold {fold + 1}")

        train_data = Dataset.from_list(train_split).map(lambda x: {'input_ids': model_tokenizer(x['altered_sentence'], truncation=True, max_length=max_length)["input_ids"], 'label': model_tokenizer(x['original_sentence'], truncation=True, max_length=max_length)["input_ids"]}, remove_columns=["original_sentence", "altered_sentence"])
        val_data = Dataset.from_list(val_split).map(lambda x: {'input_ids': model_tokenizer(x['altered_sentence'], truncation=True, max_length=max_length)["input_ids"], 'label': model_tokenizer(x['original_sentence'], truncation=True, max_length=max_length)["input_ids"]}, remove_columns=["original_sentence", "altered_sentence"])

        data_collator = DataCollatorForSeq2Seq(tokenizer=model_tokenizer)

        # loadare model
        model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

        #configuratia de antrenare.
        training_args = Seq2SeqTrainingArguments(
            output_dir=f"{model_name}_model_fold_{fold + 1}",
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            predict_with_generate=True,
            do_train=True,
            do_eval=True,
            eval_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=1,
            logging_steps=1,
            learning_rate=5e-5,
            warmup_steps=5,
            overwrite_output_dir=True,
            save_total_limit=3,
            bf16=True,
            load_best_model_at_end=True,
            push_to_hub=False,
            hub_strategy="checkpoint",
            hub_token=write_access_token,
            hub_private_repo=True,
            hub_model_id=f'{model_name}_model_fold_{fold + 1}'
        )

        trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            train_dataset=train_data,
            eval_dataset=val_data,
            tokenizer=model_tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
        )

        torch.cuda.empty_cache()
        # Execute the model training
        trainer.train()

        # Run the trained model on validation split
        eval_out = trainer.predict(val_data)
        metrics = eval_out.metrics
        print(metrics)

Map:   0%|          | 0/38610 [00:00<?, ? examples/s]

Fold 1


Map:   0%|          | 0/30888 [00:00<?, ? examples/s]

Map:   0%|          | 0/7722 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

# Gasirea parametrilor si antrenarea modelului

In runda aceasta m-am confruntat cu o eroare de tipul : CUDA out of memory, asa ca si cross-validation, gasirea parametrilor prin parameter tuning a fost mai grea din cauza resurselor limitate. Pentru gasirea parametrilor optimi pentru modelul caruia i-am dat datele procesate si tokenizate, am utilizat GridSearch CV si am definit matricea parametrilor.

Cu hyperparameter tunning am reusit sa scot un scor de

bleu        meteor   rouge
0.485928	0.606387	0.732945 pe datele de validare.

, care cred ca sunt niste metrice decente, dar care mai pot fi improved.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [4, 8, 16],
    'num_layers': [2, 4, 8],
}

random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy')
random_search.fit(train_data)
best_params = random_search.best_params_
print(best_params) # printarea celor mai buni parametrii si utilizarea acestora in modelul final

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="baseline_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=1,
    logging_steps=1,
    learning_rate=5e-5,
    warmup_steps=5,
    overwrite_output_dir=True,
    save_total_limit=3,
    bf16=True,
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_strategy="checkpoint",
    hub_token=write_access_token,
    hub_private_repo=False,
    hub_model_id='baseline_model'
)

In [ ]:
trainer = Seq2SeqTrainer( # se instantiaza trainerul de tipul
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [ ]:
torch.cuda.empty_cache()
# execute the model training
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Meteor,Rouge-l,Edit Distance
1,0.036700,0.146454,0.485928,0.606387,0.732945,34.596655


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=4827, training_loss=0.7376554240732119, metrics={'train_runtime': 3425.7835, 'train_samples_per_second': 11.27, 'train_steps_per_second': 1.409, 'total_flos': 3433899505360896.0, 'train_loss': 0.7376554240732119, 'epoch': 1.0})

# Inference

In [ ]:
# run the trained model on validation split
eval_out = trainer.predict(val_data)
metrics = eval_out.metrics
print(metrics)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'test_loss': 0.1464543342590332, 'test_bleu': 0.4859279274759491, 'test_meteor': 0.6063870403782349, 'test_rouge-L': 0.7329445885706588, 'test_edit_distance': 34.596655189375305, 'test_runtime': 226.2763, 'test_samples_per_second': 8.985, 'test_steps_per_second': 1.127}


In [ ]:
# run the trained model on custom data
test_data = [["Acest este o propizite greșita", "Aceasta este o propoziție corectă"],
             ["A Ce fdci?", "Ce faci?"],
             ["A un test scurt.", "un test scurt."]]

import pandas as pd
df = pd.DataFrame(test_data)
test_data = Dataset.from_pandas(df.rename(columns={0: "input_sentence", 1: "output_sentence"}))
test_data = test_data.map(lambda x: {'input_ids': tokenizer(x['input_sentence'], truncation=True, max_length=max_length)["input_ids"], 'label': tokenizer(x['output_sentence'], truncation=True, max_length=max_length)["input_ids"]}, remove_columns=["input_sentence", "output_sentence"])

eval_out = trainer.predict(test_data)
metrics = eval_out.metrics
print(metrics)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

{'test_loss': 4.571254730224609, 'test_bleu': 0.3521856535823236, 'test_meteor': 0.5032181571815718, 'test_rouge-L': 0.5301587301587302, 'test_edit_distance': 5.333333333333333, 'test_runtime': 0.8013, 'test_samples_per_second': 3.744, 'test_steps_per_second': 1.248}
